In [1]:
import keras

In [2]:
import tensorflow as tf

In [3]:
#MNIST: large database of handwritten digits that is commonly used for training various image processing systems
## 60,000 images, (60000,28,28)
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#Define image input dimensions. These are greyscale.
#Large images take too much time and resources.

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

#image: 28x28x1
#change channels to 3 for RGB images

In [6]:
##########################################################################
#Given input of noise (latent) vector, the Generator produces an image.

def build_generator():
    
    noise_shape = (100,) #1D array of size 100 (latent vector / noise)
    
    #Defining generator network. Here we are only using Dense layers. But networks can be complicated based
    #on the application. For example, we can use VGG (Visual Geometry Group) for super res. GAN.
    #Momentum HPO: how fast the NN trains? Gen. here takes 1dim array and generates an image of image_shape.

    model = Sequential()
    #256 units in sequential model, INPUT layer defined by input_shape. Input shape: dim. of input layer.
    #no. of units = 256 indicates units in FIRST hidden layer. We start from the first hidden layer,
    #since the input is not actually a layer
    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    
    #2nd hidden layer
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    
    #3rd hidden layer
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))
    
    #OUTPUT Final layer. np.prod gives prod. of img_shape array (28*28*1 = 784 units in output layer)
    model.add(Dense(np.prod(img_shape), activation = 'tanh'))
    #Reshape input back into dim. of img_shape
    model.add(Reshape(img_shape))    #reshapes back to (28, 28, 1).
    
    model.summary()
    
    #Our latent vector. Input returns Keras tensor. Shape indicates that expected input will be batches of 100-dim. vectors
    noise = Input(shape = noise_shape)
    
    #Generated (fake) image
    img = model(noise)

    #Indicates input noise and output img
    return Model(noise, img)

In [7]:
#Alpha — α is a hyperparameter which controls the underlying value to which the function saturates negatives network inputs.
#Momentum — Speeds up the training
##########################################################################

#Given an input image, the Discriminator outputs the likelihood of the image being real.
#Binary classification - true or false (can call it validity)

In [8]:
#Discriminator

def build_discriminator():
    
    model = Sequential()
    
    #Define Input layer:
    model.add(Flatten(input_shape = img_shape))
    
    #First hidden layer:
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    
    #Second hidden layer:
    model.add(Dense(256))
    model.add(LeakyReLU(alpha = 0.2))
    
    #Output layer:
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.summary()
    
    img = Input(shape = img_shape)
    validity = model(img)
    #returns supplied image img, and validity value (real or fake)
    return Model(img, validity)
#The validity is the Discriminator’s guess of input being real or not.

In [9]:
#Now that we have constructed our two models it’s time to pit them against each other.
#We do this by defining a training function, loading the data set, re-scaling our training images
#and setting the ground truths

#Just for understanding
(X_training, _), (_, _) = mnist.load_data()
print("X_train shape:",  np.shape(X_training), ", 60,000 images of shape 28x28")
X_training = np.expand_dims(X_training, axis = 3)
print("After adding extra dimension for channel", np.shape(X_training))
print("X_train[i] gives ith image: ", np.shape(X_train[59999]))
print(X_train[0][0][0])    #1 [0]: 0th image,  [0][0]: (28x1), [0][0][0]: (1,), [0][0][0][0]: value in x1 column
print("X_train type: ", type(X_train))

In [12]:
#Just for understanding
indexx = np.random.randint(0, 60000, 25) #rand ints from [0, 60000) of (half_batch) samples
print("Shape of indexx: ", np.shape(indexx))
testt = X_training[indexx]
print("Shape of testt: ",np.shape(testt))

noisee = np.random.normal(0, 1, (25, 100))
print(np.shape(noisee))
valid_yyy = np.array([1] * 25)
print(np.shape(valid_yyy))
print(valid_yyy)

Shape of indexx:  (25,)
Shape of testt:  (25, 28, 28, 1)
(25, 100)
(25,)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [13]:
def train(epochs, batch_size, save_interval = 50):
    
    #load the real dataset
    (X_train, _), (_, _) = mnist.load_data()   #(60000, 28, 28)
    
    #Convert to float and Rescale -1 to 1 (can also do 0 to 1)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5         #cast panda object to float type (using .astype())
    
    #Add channels dimension as the input to our gen. and disc. has a shape 28x28x1.
    X_train = np.expand_dims(X_train, axis = 3)   #(60000, 28, 28, 1)
    
    #half of the batch are fake and other half are real, which go into the discriminator
    half_batch = int(batch_size / 2)
    
    #We then loop through a number of epochs to train our Discriminator by first selecting
    #a random batch of images from our true dataset, generating a set of images from our
    #Generator, feeding both set of images into our Discriminator, and finally setting the
    #loss parameters for both the real and fake images, as well as the combined loss.
    
    for epoch in range(epochs):
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # Select a random half batch of real images
        idx = np.random.randint(0, X_train.shape[0], half_batch) #rand ints from [0, 60000) of (half_batch) samples
        imgs = X_train[idx]  #img is an array of (half_batch) no. of REAL images 28x28x1 dim.
        
        noise = np.random.normal(0, 1, (half_batch, 100))  #rand normal from [0, 1) of (half_batch * 100) samples
        
        # Generate a half batch of fake images. Input (half_batch) no. of latent noise vectors,
        #gives (half_batch) FAKE image outputs.
        gen_imgs = generator.predict(noise)
        
        #Train the discriminator on real and fake images, separately
        #Research showed that separate training is more effective.
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))  #target data: np.ones((half_batch, 1)),
        #ones corresponding to real images, each REAL image corresponds to a value of 1
        
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        #zeros corresponding to fake images. Each FAKE image corresponds to a value of 0. gen_imgs generated from noise.
        
        #take average loss from real and fake images. 
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        #And within the same loop we train our Generator, by setting the input noise and
        #ultimately training the Generator to have the Discriminator label its samples as valid
        #by specifying the gradient loss.
        
        # ---------------------
        #  Train Generator
        # ---------------------
        
        #Create noise vectors as input for generator. 
        #Create as many noise vectors as defined by the batch size. 
        #Based on normal distribution. Output will be of size (batch size, 100)
        noise = np.random.normal(0, 1, (batch_size, 100))
        
        #The generator wants the discriminator to label the generated samples as valid (ones)
        #This is where the generator is trying to trick discriminator into believing that
        #the generated image is true (hence value of 1 for y)
        valid_y = np.array([1] * batch_size)                 #Creates an array of all ones of size = batch size
        
        
        # Generator is part of the 'combined' where it is directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator does a great
        #job of fooling the discriminator, then the output will be 1 (true). The discriminator in the 'combined' 
        #is not being trained here while the generator is being trained.
        g_loss = combined.train_on_batch(noise, valid_y)
        
        #Additionally, in order for us to keep track of our training process, we print the
        #progress and save the sample image output depending on the epoch interval specified.  
        #Plot the progress
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
        
        #If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [14]:
def save_imgs(epoch):
    r, c = 5, 5    # 5x5 grid of images
    noise = np.random.normal(0, 1, (r * c, 100))   #noise array of 25 latent vectors with 100 elements in each.
    gen_imgs = generator.predict(noise)
    
    #Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap = 'gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()
    
#This function saves our images for us to view.
#Generates 25 random images and saving. 5x5 grid of 25 images

In [15]:
##############################################################################

#Define our optimizer for easy use later on.
#That way if you change your mind, you can change it easily here
optimizer = Adam(0.0002, 0.5)    #Learning rate and momentum.

#Build and compile the discriminator first. 
#Generator will be trained as part of the combined model, later. 
#pick the loss function and the type of metric to keep track.                 
#Binary cross entropy as we are doing prediction and it is a better loss function compared to MSE or other. 

discriminator = build_discriminator()
discriminator.compile(loss = 'binary_crossentropy',
                     optimizer = optimizer,
                     metrics = ['accuracy'])

#Build and compile our Discriminator, pick the loss function

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

In [16]:
#Since we are only generating (faking) images, let us not track any metrics.
generator = build_generator()
generator.compile(loss = 'binary_crossentropy', optimizer = optimizer)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [17]:
#This builds the Generator and defines the input noise. 
#In a GAN, the Generator network takes noise z as an input to produce its images.

z = Input(shape = (100, ))
img = generator(z)

#This ensures that when we combine our networks we only train the Generator.
#While generator training we do not want discriminator weights to be adjusted. 
#This Doesn't affect the above descriminator training. 
discriminator.trainable = False

#This specifies that our Discriminator will take the images generated by our Generator
#and true dataset and set its output to a parameter called valid, which will indicate
#whether the input is real or not.  
valid = discriminator(img)        #Validity check on the generated image (prob. of image being REAL)

In [18]:
%%time
#Here we combined the models and also set our loss function and optimizer. 
#Again, we are only training the generator here. 
#The ultimate goal here is for the Generator to fool the Discriminator.  
#The combined model  (stacked generator and discriminator) takes
#noise as input => generates images => determines validity

combined = Model(z, valid)   #noise input vector, output validity (real or fake) for this combined model.
combined.compile(loss = 'binary_crossentropy', optimizer = optimizer)

#Call train function for training
train(epochs = 5000, batch_size = 32, save_interval = 10)

0 [D loss: 0.539142, acc.: 87.50%] [G loss: 0.638050]
1 [D loss: 0.331120, acc.: 100.00%] [G loss: 0.661910]
2 [D loss: 0.321517, acc.: 87.50%] [G loss: 0.734603]
3 [D loss: 0.351299, acc.: 84.38%] [G loss: 0.827448]
4 [D loss: 0.309650, acc.: 90.62%] [G loss: 0.926525]
5 [D loss: 0.259354, acc.: 100.00%] [G loss: 1.081645]
6 [D loss: 0.238601, acc.: 100.00%] [G loss: 1.236523]
7 [D loss: 0.213295, acc.: 100.00%] [G loss: 1.526788]
8 [D loss: 0.172382, acc.: 100.00%] [G loss: 1.663885]
9 [D loss: 0.115962, acc.: 100.00%] [G loss: 1.841275]
10 [D loss: 0.107671, acc.: 100.00%] [G loss: 2.023177]
11 [D loss: 0.100504, acc.: 100.00%] [G loss: 2.138030]
12 [D loss: 0.086795, acc.: 100.00%] [G loss: 2.295486]
13 [D loss: 0.066132, acc.: 100.00%] [G loss: 2.313000]
14 [D loss: 0.077654, acc.: 100.00%] [G loss: 2.588876]
15 [D loss: 0.044550, acc.: 100.00%] [G loss: 2.641369]
16 [D loss: 0.059249, acc.: 100.00%] [G loss: 2.648496]
17 [D loss: 0.054858, acc.: 100.00%] [G loss: 2.716866]
18 [D 

147 [D loss: 0.093458, acc.: 100.00%] [G loss: 4.306538]
148 [D loss: 0.840996, acc.: 59.38%] [G loss: 2.402052]
149 [D loss: 0.238810, acc.: 93.75%] [G loss: 3.268382]
150 [D loss: 0.041227, acc.: 100.00%] [G loss: 4.005667]
151 [D loss: 0.251170, acc.: 87.50%] [G loss: 2.587433]
152 [D loss: 0.321223, acc.: 81.25%] [G loss: 3.533769]
153 [D loss: 0.125330, acc.: 96.88%] [G loss: 3.687576]
154 [D loss: 0.070035, acc.: 100.00%] [G loss: 3.600757]
155 [D loss: 0.082985, acc.: 100.00%] [G loss: 3.560668]
156 [D loss: 0.205434, acc.: 93.75%] [G loss: 2.674193]
157 [D loss: 0.109120, acc.: 93.75%] [G loss: 2.910315]
158 [D loss: 0.172174, acc.: 93.75%] [G loss: 2.652829]
159 [D loss: 0.197370, acc.: 96.88%] [G loss: 3.478486]
160 [D loss: 0.328827, acc.: 90.62%] [G loss: 3.216956]
161 [D loss: 0.065923, acc.: 100.00%] [G loss: 3.299154]
162 [D loss: 0.148875, acc.: 100.00%] [G loss: 2.863734]
163 [D loss: 0.148191, acc.: 93.75%] [G loss: 2.702619]
164 [D loss: 0.166294, acc.: 90.62%] [G lo

294 [D loss: 0.678245, acc.: 53.12%] [G loss: 1.129620]
295 [D loss: 0.822463, acc.: 37.50%] [G loss: 0.787509]
296 [D loss: 0.728502, acc.: 46.88%] [G loss: 0.899027]
297 [D loss: 0.771367, acc.: 40.62%] [G loss: 0.836403]
298 [D loss: 0.677251, acc.: 59.38%] [G loss: 0.885635]
299 [D loss: 0.767870, acc.: 40.62%] [G loss: 0.773119]
300 [D loss: 0.857680, acc.: 43.75%] [G loss: 0.789968]
301 [D loss: 0.718039, acc.: 40.62%] [G loss: 0.788478]
302 [D loss: 0.763638, acc.: 37.50%] [G loss: 0.743357]
303 [D loss: 0.642071, acc.: 59.38%] [G loss: 0.940871]
304 [D loss: 0.702223, acc.: 37.50%] [G loss: 0.865090]
305 [D loss: 0.794336, acc.: 37.50%] [G loss: 0.760289]
306 [D loss: 0.634040, acc.: 50.00%] [G loss: 0.835119]
307 [D loss: 0.795958, acc.: 40.62%] [G loss: 0.659304]
308 [D loss: 0.815355, acc.: 25.00%] [G loss: 0.598451]
309 [D loss: 0.732475, acc.: 46.88%] [G loss: 0.720768]
310 [D loss: 0.702026, acc.: 43.75%] [G loss: 0.741574]
311 [D loss: 0.643057, acc.: 56.25%] [G loss: 0.

441 [D loss: 0.665690, acc.: 50.00%] [G loss: 0.715878]
442 [D loss: 0.710309, acc.: 34.38%] [G loss: 0.716495]
443 [D loss: 0.650003, acc.: 43.75%] [G loss: 0.698837]
444 [D loss: 0.688517, acc.: 40.62%] [G loss: 0.673890]
445 [D loss: 0.665928, acc.: 50.00%] [G loss: 0.650871]
446 [D loss: 0.669651, acc.: 59.38%] [G loss: 0.685512]
447 [D loss: 0.686689, acc.: 37.50%] [G loss: 0.710536]
448 [D loss: 0.709554, acc.: 53.12%] [G loss: 0.693283]
449 [D loss: 0.693941, acc.: 34.38%] [G loss: 0.702464]
450 [D loss: 0.642616, acc.: 50.00%] [G loss: 0.687447]
451 [D loss: 0.659843, acc.: 50.00%] [G loss: 0.637179]
452 [D loss: 0.644322, acc.: 53.12%] [G loss: 0.655027]
453 [D loss: 0.697570, acc.: 43.75%] [G loss: 0.669111]
454 [D loss: 0.644173, acc.: 62.50%] [G loss: 0.641891]
455 [D loss: 0.683197, acc.: 59.38%] [G loss: 0.696786]
456 [D loss: 0.703910, acc.: 56.25%] [G loss: 0.704758]
457 [D loss: 0.652337, acc.: 53.12%] [G loss: 0.736109]
458 [D loss: 0.674830, acc.: 56.25%] [G loss: 0.

590 [D loss: 0.688072, acc.: 43.75%] [G loss: 0.685856]
591 [D loss: 0.689541, acc.: 40.62%] [G loss: 0.673493]
592 [D loss: 0.659028, acc.: 50.00%] [G loss: 0.678998]
593 [D loss: 0.665319, acc.: 56.25%] [G loss: 0.662200]
594 [D loss: 0.639338, acc.: 62.50%] [G loss: 0.717083]
595 [D loss: 0.662978, acc.: 59.38%] [G loss: 0.726961]
596 [D loss: 0.659186, acc.: 68.75%] [G loss: 0.719665]
597 [D loss: 0.629736, acc.: 68.75%] [G loss: 0.748731]
598 [D loss: 0.739534, acc.: 40.62%] [G loss: 0.706321]
599 [D loss: 0.698224, acc.: 56.25%] [G loss: 0.704787]
600 [D loss: 0.680330, acc.: 56.25%] [G loss: 0.690944]
601 [D loss: 0.661623, acc.: 56.25%] [G loss: 0.742668]
602 [D loss: 0.666626, acc.: 59.38%] [G loss: 0.731474]
603 [D loss: 0.678139, acc.: 34.38%] [G loss: 0.704168]
604 [D loss: 0.640259, acc.: 56.25%] [G loss: 0.688170]
605 [D loss: 0.668959, acc.: 59.38%] [G loss: 0.709041]
606 [D loss: 0.673946, acc.: 56.25%] [G loss: 0.728322]
607 [D loss: 0.646574, acc.: 62.50%] [G loss: 0.

...

4359 [D loss: 0.599781, acc.: 75.00%] [G loss: 0.939216]
4360 [D loss: 0.578686, acc.: 71.88%] [G loss: 1.011243]
4361 [D loss: 0.550333, acc.: 75.00%] [G loss: 0.989427]
4362 [D loss: 0.535910, acc.: 68.75%] [G loss: 0.982107]
4363 [D loss: 0.615198, acc.: 53.12%] [G loss: 1.038564]
4364 [D loss: 0.597509, acc.: 65.62%] [G loss: 0.948614]
4365 [D loss: 0.518723, acc.: 84.38%] [G loss: 0.983664]
4366 [D loss: 0.710210, acc.: 56.25%] [G loss: 0.929185]
4367 [D loss: 0.635114, acc.: 62.50%] [G loss: 0.925799]
4368 [D loss: 0.654882, acc.: 59.38%] [G loss: 0.893176]
4369 [D loss: 0.588314, acc.: 68.75%] [G loss: 0.896442]
4370 [D loss: 0.651935, acc.: 56.25%] [G loss: 0.871426]
4371 [D loss: 0.599635, acc.: 65.62%] [G loss: 0.895784]
4372 [D loss: 0.608287, acc.: 59.38%] [G loss: 0.972723]
4373 [D loss: 0.610999, acc.: 65.62%] [G loss: 1.015918]
4374 [D loss: 0.629077, acc.: 59.38%] [G loss: 1.051726]
4375 [D loss: 0.629373, acc.: 59.38%] [G loss: 0.984630]
4376 [D loss: 0.657456, acc.: 6

4503 [D loss: 0.652148, acc.: 65.62%] [G loss: 0.981075]
4504 [D loss: 0.574849, acc.: 71.88%] [G loss: 0.986132]
4505 [D loss: 0.639205, acc.: 62.50%] [G loss: 1.042176]
4506 [D loss: 0.688618, acc.: 56.25%] [G loss: 0.846367]
4507 [D loss: 0.669475, acc.: 68.75%] [G loss: 0.953508]
4508 [D loss: 0.582450, acc.: 78.12%] [G loss: 0.865133]
4509 [D loss: 0.550958, acc.: 78.12%] [G loss: 0.931794]
4510 [D loss: 0.594598, acc.: 75.00%] [G loss: 0.909379]
4511 [D loss: 0.707349, acc.: 50.00%] [G loss: 0.884539]
4512 [D loss: 0.670288, acc.: 53.12%] [G loss: 0.891657]
4513 [D loss: 0.775862, acc.: 56.25%] [G loss: 0.889810]
4514 [D loss: 0.709686, acc.: 53.12%] [G loss: 0.933919]
4515 [D loss: 0.660893, acc.: 59.38%] [G loss: 0.905688]
4516 [D loss: 0.627170, acc.: 65.62%] [G loss: 0.841599]
4517 [D loss: 0.564781, acc.: 78.12%] [G loss: 0.868681]
4518 [D loss: 0.639313, acc.: 65.62%] [G loss: 0.960133]
4519 [D loss: 0.641253, acc.: 59.38%] [G loss: 1.000335]
4520 [D loss: 0.647698, acc.: 6

4648 [D loss: 0.737797, acc.: 50.00%] [G loss: 0.879928]
4649 [D loss: 0.584728, acc.: 62.50%] [G loss: 0.841141]
4650 [D loss: 0.665385, acc.: 65.62%] [G loss: 0.882649]
4651 [D loss: 0.675487, acc.: 65.62%] [G loss: 0.847121]
4652 [D loss: 0.618483, acc.: 65.62%] [G loss: 0.859950]
4653 [D loss: 0.745586, acc.: 46.88%] [G loss: 0.869379]
4654 [D loss: 0.667325, acc.: 53.12%] [G loss: 0.884676]
4655 [D loss: 0.620707, acc.: 65.62%] [G loss: 0.799538]
4656 [D loss: 0.660695, acc.: 53.12%] [G loss: 0.939035]
4657 [D loss: 0.664972, acc.: 59.38%] [G loss: 0.825197]
4658 [D loss: 0.606832, acc.: 65.62%] [G loss: 0.887299]
4659 [D loss: 0.584504, acc.: 71.88%] [G loss: 0.829526]
4660 [D loss: 0.563804, acc.: 71.88%] [G loss: 0.871118]
4661 [D loss: 0.587404, acc.: 68.75%] [G loss: 0.935848]
4662 [D loss: 0.717713, acc.: 56.25%] [G loss: 1.000694]
4663 [D loss: 0.580422, acc.: 75.00%] [G loss: 0.916605]
4664 [D loss: 0.734745, acc.: 46.88%] [G loss: 0.913547]
4665 [D loss: 0.594091, acc.: 6

4793 [D loss: 0.528328, acc.: 84.38%] [G loss: 0.926757]
4794 [D loss: 0.642371, acc.: 56.25%] [G loss: 0.931136]
4795 [D loss: 0.587856, acc.: 65.62%] [G loss: 0.993762]
4796 [D loss: 0.675248, acc.: 59.38%] [G loss: 0.963643]
4797 [D loss: 0.743275, acc.: 50.00%] [G loss: 0.856921]
4798 [D loss: 0.591819, acc.: 65.62%] [G loss: 0.888137]
4799 [D loss: 0.597720, acc.: 78.12%] [G loss: 0.933868]
4800 [D loss: 0.632078, acc.: 68.75%] [G loss: 0.932098]
4801 [D loss: 0.701068, acc.: 40.62%] [G loss: 0.826792]
4802 [D loss: 0.495242, acc.: 87.50%] [G loss: 0.888604]
4803 [D loss: 0.714794, acc.: 50.00%] [G loss: 0.891784]
4804 [D loss: 0.712416, acc.: 53.12%] [G loss: 0.965383]
4805 [D loss: 0.592073, acc.: 75.00%] [G loss: 1.029040]
4806 [D loss: 0.609823, acc.: 68.75%] [G loss: 0.936797]
4807 [D loss: 0.644701, acc.: 68.75%] [G loss: 0.927622]
4808 [D loss: 0.580666, acc.: 71.88%] [G loss: 0.947604]
4809 [D loss: 0.600263, acc.: 62.50%] [G loss: 0.900058]
4810 [D loss: 0.722255, acc.: 5

4937 [D loss: 0.665637, acc.: 62.50%] [G loss: 0.973478]
4938 [D loss: 0.693974, acc.: 56.25%] [G loss: 0.974314]
4939 [D loss: 0.655521, acc.: 56.25%] [G loss: 0.972152]
4940 [D loss: 0.634159, acc.: 78.12%] [G loss: 1.122934]
4941 [D loss: 0.587270, acc.: 65.62%] [G loss: 0.974106]
4942 [D loss: 0.706232, acc.: 65.62%] [G loss: 0.911774]
4943 [D loss: 0.688234, acc.: 59.38%] [G loss: 0.901940]
4944 [D loss: 0.545024, acc.: 78.12%] [G loss: 0.910432]
4945 [D loss: 0.638832, acc.: 62.50%] [G loss: 0.997380]
4946 [D loss: 0.588175, acc.: 68.75%] [G loss: 0.999083]
4947 [D loss: 0.646445, acc.: 71.88%] [G loss: 1.002096]
4948 [D loss: 0.681322, acc.: 59.38%] [G loss: 0.906281]
4949 [D loss: 0.616712, acc.: 56.25%] [G loss: 0.978551]
4950 [D loss: 0.632493, acc.: 62.50%] [G loss: 0.907012]
4951 [D loss: 0.586970, acc.: 59.38%] [G loss: 0.853008]
4952 [D loss: 0.558072, acc.: 78.12%] [G loss: 0.883196]
4953 [D loss: 0.565319, acc.: 75.00%] [G loss: 0.951855]
4954 [D loss: 0.730831, acc.: 4

In [19]:
#Save model for future use to generate fake images

generator.save('generator_model.h5')   #Test the model on GAN4_predict
#Change epochs back to 30K

#Epochs dictate the number of backward and forward propagations, the batch_size
#indicates the number of training samples per backward/forward propagation, and the
#sample_interval specifies after how many epochs we call our sample_image function.